*Packages and initial settings*

In [1]:
# Load packages
using FileIO;
using DataFrames;
using Colors, PlotlyJS;
using ORCA, PlotlyBase;
using Dates;

# Colors
c1 = "rgba(0, 48, 158, .75)";
c2 = "rgba(255, 0, 0, .75)";
c3 = "rgba(255, 190, 0, .75)";

titles = ["CBO cycle: GDP", "Real GDP", "SPF: Real GDP", "Unemployment rate", "Employment", "Oil price", "CPI inflation", "SPF: Expected inflation", "UoM: Expected inflation"];
scales = ["Bil. Chn. 2009\$", "Bil. Chn. 2009\$", "Bil. Chn. 2009\$", "Percent", "Thous.", "\$/Barrel", "Percent", "Percent", "Percent"];

# Baseline csv output folder
baseline_path = "./models/baseline_oos/results_csv";

# Restricted csv output folder
restricted_path = "./models/restricted_oos/results_csv";

HTML{String}("<script>\n// Immediately-invoked-function-expression to avoid global variables.\n(function() {\n    var warning_div = document.getElementById(\"webio-warning-13903353666533876976\");\n    var hide = function () {\n        var script = document.getElementById(\"webio-setup-4567377296650493489\");\n        var parent = script && script.parentElement;\n        var grandparent = parent && parent.parentElement;\n        if (grandparent) {\n            grandparent.style.display = \"none\";\n        }\n        warning_div.style.display = \"none\";\n    };\n    if (typeof Jupyter !== \"undefined\") {\n        console.log(\"WebIO detected Jupyter notebook environment.\");\n        // Jupyter notebook.\n        var extensions = (\n            Jupyter\n            && Jupyter.notebook.config.data\n            && Jupyter.notebook.config.data.load_extensions\n        );\n        if (extensions && extensions[\"webio-jupyter-notebook\"]) {\n            // Extension already loaded.\n            console.log(\"Jupyter WebIO nbextension detected; not loading ad-hoc.\");\n            hide();\n            return;\n        }\n    } else if (window.location.pathname.includes(\"/lab\")) {\n        // Guessing JupyterLa\n        console.log(\"Jupyter Lab detected; make sure the @webio/jupyter-lab-provider labextension is installed.\");\n        hide();\n        return;\n    }\n})();\n\n</script>\n<p\n    id=\"webio-warning-13903353666533876976\"\n    class=\"output_text output_stderr\"\n    style=\"padding: 1em; font-weight: bold;\"\n>\n    Unable to load WebIO. Please make sure WebIO works for your Jupyter client.\n    For troubleshooting, please see <a href=\"https://juliagizmos.github.io/WebIO.jl/latest/providers/ijulia/\">\n    the WebIO/IJulia documentation</a>.\n    <!-- TODO: link to installation docs. -->\n</p>\n")

*Load MSFE output*

In [2]:
# Baseline
baseline_msfe = DataFrame(load("$(baseline_path)/baseline_msfe_tc.csv"));
baseline_rw_msfe = DataFrame(load("$(baseline_path)/baseline_msfe_rw.csv"));
rename!(baseline_msfe, [:GDP, :GDP_SPF, :URATE, :EMPL, :OIL, :INFL, :INFL_SPF, :EXP_INFL]);
rename!(baseline_rw_msfe, [:GDP, :GDP_SPF, :URATE, :EMPL, :OIL, :INFL, :INFL_SPF, :EXP_INFL]);

# Restricted
restricted_msfe = DataFrame(load("$(restricted_path)/restricted_msfe_tc.csv"));
restricted_rw_msfe = DataFrame(load("$(restricted_path)/restricted_msfe_rw.csv"));
rename!(restricted_msfe, [:CBO, :GDP, :GDP_SPF, :URATE, :EMPL, :OIL, :INFL, :INFL_SPF, :EXP_INFL]);
rename!(restricted_rw_msfe, [:CBO, :GDP, :GDP_SPF, :URATE, :EMPL, :OIL, :INFL, :INFL_SPF, :EXP_INFL]);

*Load output gap and potential output*

In [3]:
# Baseline
baseline_output_gap_df = DataFrame(load("$(baseline_path)/baseline_output_gap.csv"));
baseline_potential_yoy_df = DataFrame(load("$(baseline_path)/baseline_potential_output_yoy.csv"));

# Restricted
restricted_output_gap_df = DataFrame(load("$(restricted_path)/restricted_output_gap.csv"));
restricted_potential_yoy_df = DataFrame(load("$(restricted_path)/restricted_potential_output_yoy.csv"));

# Last common release
common_releases = intersect([Date(release) for release in names(baseline_output_gap_df)[2:end]], 
                            [Date(release) for release in names(restricted_output_gap_df)[2:end]]);

last_common_release = maximum(common_releases);

*Load (one-step ahead) forecast output*

*1. Real GDP*

In [4]:
# Horizon
gdp_hz = 3;

# Baseline
baseline_gdp_forecast_df = DataFrame(load("$(baseline_path)/baseline_forecast_gdp.csv"));
baseline_gdp_forecast_df = DataFrame(releases = names(baseline_gdp_forecast_df)[2:end], baseline = collect(baseline_gdp_forecast_df[gdp_hz, 2:end]));

# Restricted
restricted_gdp_forecast_df = DataFrame(load("$(restricted_path)/restricted_forecast_gdp.csv"));
restricted_gdp_forecast_df = DataFrame(releases = names(restricted_gdp_forecast_df)[2:end], restricted = collect(restricted_gdp_forecast_df[gdp_hz, 2:end]));

# Outer join
gdp_forecast_df = outerjoin(baseline_gdp_forecast_df, restricted_gdp_forecast_df, on=:releases);
sort!(gdp_forecast_df, :releases);

# Fill-in the missing observations in baseline (corresponding to the "restricted" releases)
for t=1:size(gdp_forecast_df,1)
    if ismissing(gdp_forecast_df[t,:baseline]) && t > 1
        gdp_forecast_df[t,:baseline] = gdp_forecast_df[t-1,:baseline];
    end
end

# Load outturn
gdp_outturn_df = DataFrame(load("$(restricted_path)/restricted_outturn_gdp.csv"));
gdp_outturn_df = DataFrame(releases = names(gdp_outturn_df)[2:end], outturn = collect(gdp_outturn_df[gdp_hz, 2:end]));
sort!(gdp_outturn_df, :releases);

# Add outturn to forecast_df
gdp_forecast_df = outerjoin(gdp_forecast_df, gdp_outturn_df, on=:releases);

# Convert cols to Array{Float64}
gdp_forecast_df[!, :baseline] = convert(Array{Float64}, gdp_forecast_df[!,:baseline]);
gdp_forecast_df[!, :restricted] = convert(Array{Float64}, gdp_forecast_df[!,:restricted]);
gdp_forecast_df[!, :outturn] = convert(Array{Float64}, gdp_forecast_df[!,:outturn]);

In [5]:
# Horizon
headline_hz = 1;

# Baseline
baseline_headline_forecast_df = DataFrame(load("$(baseline_path)/baseline_forecast_headline.csv"));
baseline_headline_forecast_df = DataFrame(releases = names(baseline_headline_forecast_df)[2:end], baseline = collect(baseline_headline_forecast_df[headline_hz, 2:end]));

# Restricted
restricted_headline_forecast_df = DataFrame(load("$(restricted_path)/restricted_forecast_headline.csv"));
restricted_headline_forecast_df = DataFrame(releases = names(restricted_headline_forecast_df)[2:end], restricted = collect(restricted_headline_forecast_df[headline_hz, 2:end]));

# Outer join
headline_forecast_df = outerjoin(baseline_headline_forecast_df, restricted_headline_forecast_df, on=:releases);
sort!(headline_forecast_df, :releases);

# Fill-in the missing observations in baseline (corresponding to the "restricted"restricted releases)
for t=1:size(headline_forecast_df,1)
    if ismissing(headline_forecast_df[t,:baseline]) && t > 1
        headline_forecast_df[t,:baseline] = headline_forecast_df[t-1,:baseline];
    end
end

# Load outturn
headline_outturn_df = DataFrame(load("$(restricted_path)/restricted_outturn_headline.csv"));
headline_outturn_df = DataFrame(releases = names(headline_outturn_df)[2:end], outturn = collect(headline_outturn_df[headline_hz, 2:end]));
sort!(headline_outturn_df, :releases);

# Add outturn to forecast_df
headline_forecast_df = outerjoin(headline_forecast_df, headline_outturn_df, on=:releases);

# Convert cols to Array{Float64}
headline_forecast_df[!, :baseline] = convert(Array{Float64}, headline_forecast_df[!,:baseline]);
headline_forecast_df[!, :restricted] = convert(Array{Float64}, headline_forecast_df[!,:restricted]);
headline_forecast_df[!, :outturn] = convert(Array{Float64}, headline_forecast_df[!,:outturn]);

*Forecasting evaluation charts*

In [6]:
figure = Array{Any}(undef, 9);

for i=1:9
    
    # i-th data
    ith_restricted_data = collect(skipmissing(restricted_msfe[:,i]));

    # CBO cycle is not used in the baseline model
    if i == 1
        ith_baseline_data = NaN*ones(length(ith_restricted_data));
    else
        ith_baseline_data = collect(skipmissing(baseline_msfe[:,i-1]));
    end
    
    # x-axis
    if length(ith_restricted_data) == 12
        xx = collect(1:1:12);
        xx_title = "Quarters";
    else
        xx = collect(1:1:36);
        xx_title = "Months";
    end
    
    # Chart
    trace1 = scatter(x=xx, y=ith_baseline_data, line=attr(width=1.4, color=c1), marker_symbol="x", mode="lines+markers", showlegend=i==2, name="Baseline");
    trace2 = scatter(x=xx, y=ith_restricted_data, line=attr(width=1.4, color=c2), marker_symbol="x", mode="lines+markers", showlegend=i==2, name="Restricted");
    
    layout = Layout(title=titles[i], titlefont_size=12,
                    xaxis=attr(tickfont_size=10, showgrid=true, linecolor="black", mirror=true, nticks=10, tickangle=0, range=[minimum(xx), maximum(xx)], titlefont=attr(size=10), title=xx_title),
                    yaxis=attr(zeroline=false, tickfont_size=10, showgrid=true, linecolor="black", nticks=10, mirror=true, titlefont=attr(size=10), title=scales[i]));
        
    figure[i] = plot([trace1, trace2], layout);
end

fig = [figure[1] figure[2] figure[3]; figure[4] figure[5] figure[6]; figure[7] figure[8] figure[9]];

# Size
fig.plot.layout["width"]  = 1000;
fig.plot.layout["height"] = 800;

# Margins
fig.plot.layout["margin"][:b]  = 40;
fig.plot.layout["margin"][:t]  = 40;
fig.plot.layout["margin"][:r]  = 40;
fig.plot.layout["margin"][:l]  = 40;

# Title size
for i=1:9
    fig.plot.layout["annotations"][i][:font][:size] = 12;
end

fig.plot.layout["legend"] = attr(y=-0.08, x=0.403, font=attr(size=10), orientation="h")

PlotlyBase.savefig(fig, "./annex/MSFE.pdf", format="pdf")

"./annex/MSFE.pdf"

In [7]:
figure = Array{Any}(undef, 9);

for i=1:9
    
    # i-th data
    ith_restricted_data = collect(skipmissing(restricted_msfe[:,i]./restricted_rw_msfe[:,i]));

    # CBO cycle is not used in the baseline model
    if i == 1
        ith_baseline_data = NaN*ones(length(ith_restricted_data));
    else
        ith_baseline_data = collect(skipmissing(baseline_msfe[:,i-1]./baseline_rw_msfe[:,i-1]));
    end
    
    # x-axis
    if length(ith_restricted_data) == 12
        xx = collect(1:1:12);
        xx_title = "Quarters";
    else
        xx = collect(1:1:36);
        xx_title = "Months";
    end
    
    # Chart
    trace1 = scatter(x=xx, y=ith_baseline_data, line=attr(width=1.4, color=c1), marker_symbol="x", mode="lines+markers", showlegend=i==2, name="Baseline");
    trace2 = scatter(x=xx, y=ith_restricted_data, line=attr(width=1.4, color=c2), marker_symbol="x", mode="lines+markers", showlegend=i==2, name="Restricted");
    trace3 = scatter(x=xx, y=ones(length(xx)), line=attr(width=1.4, color="black", dash="dot"), mode="lines", showlegend=false);
    
    layout = Layout(title=titles[i], titlefont_size=12,
                    xaxis=attr(tickfont_size=10, showgrid=true, linecolor="black", mirror=true, nticks=10, tickangle=0, range=[minimum(xx), maximum(xx)], titlefont=attr(size=10), title=xx_title),
                    yaxis=attr(zeroline=false, tickfont_size=10, showgrid=true, linecolor="black", nticks=10, mirror=true, titlefont=attr(size=10), title="Model relative to RW"));
        
    figure[i] = plot([trace1, trace2, trace3], layout);
end

fig = [figure[1] figure[2] figure[3]; figure[4] figure[5] figure[6]; figure[7] figure[8] figure[9]];

# Size
fig.plot.layout["width"]  = 1000;
fig.plot.layout["height"] = 800;

# Margins
fig.plot.layout["margin"][:b]  = 40;
fig.plot.layout["margin"][:t]  = 40;
fig.plot.layout["margin"][:r]  = 40;
fig.plot.layout["margin"][:l]  = 40;

# Title size
for i=1:9
    fig.plot.layout["annotations"][i][:font][:size] = 12;
end

fig.plot.layout["legend"] = attr(y=-0.08, x=0.403, font=attr(size=10), orientation="h")

PlotlyBase.savefig(fig, "./annex/RMSFE.pdf", format="pdf")

"./annex/RMSFE.pdf"

In [8]:
figure = Array{Any}(undef, 9);

for i=1:9
    
    # i-th data
    ith_restricted_data = collect(skipmissing(restricted_msfe[:,i]./restricted_rw_msfe[:,i]));

    # CBO cycle is not used in the baseline model
    if i == 1
        ith_baseline_data = NaN*ones(length(ith_restricted_data));
    else
        ith_baseline_data = collect(skipmissing(baseline_msfe[:,i-1]./restricted_rw_msfe[:,i]));
    end
    
    # x-axis
    if length(ith_restricted_data) == 12
        xx = collect(1:1:12);
        xx_title = "Quarters";
    else
        xx = collect(1:1:36);
        xx_title = "Months";
    end
    
    # Chart
    trace1 = scatter(x=xx, y=ith_baseline_data, line=attr(width=1.4, color=c1), marker_symbol="x", mode="lines+markers", showlegend=i==2, name="Baseline");
    trace2 = scatter(x=xx, y=ith_restricted_data, line=attr(width=1.4, color=c2), marker_symbol="x", mode="lines+markers", showlegend=i==2, name="Restricted");
    trace3 = scatter(x=xx, y=ones(length(xx)), line=attr(width=1.4, color="black", dash="dot"), mode="lines", showlegend=false);
    
    layout = Layout(title=titles[i], titlefont_size=12,
                    xaxis=attr(tickfont_size=10, showgrid=true, linecolor="black", mirror=true, nticks=10, tickangle=0, range=[minimum(xx), maximum(xx)], titlefont=attr(size=10), title=xx_title),
                    yaxis=attr(zeroline=false, tickfont_size=10, showgrid=true, linecolor="black", nticks=10, mirror=true, titlefont=attr(size=10), title="Model relative to RW"));
        
    figure[i] = plot([trace1, trace2, trace3], layout);
end

fig = [figure[1] figure[2] figure[3]; figure[4] figure[5] figure[6]; figure[7] figure[8] figure[9]];

# Size
fig.plot.layout["width"]  = 1000;
fig.plot.layout["height"] = 800;

# Margins
fig.plot.layout["margin"][:b]  = 40;
fig.plot.layout["margin"][:t]  = 40;
fig.plot.layout["margin"][:r]  = 40;
fig.plot.layout["margin"][:l]  = 40;

# Title size
for i=1:9
    fig.plot.layout["annotations"][i][:font][:size] = 12;
end

fig.plot.layout["legend"] = attr(y=-0.08, x=0.403, font=attr(size=10), orientation="h")

PlotlyBase.savefig(fig, "./annex/RMSFE_restricted_rw.pdf", format="pdf")

"./annex/RMSFE_restricted_rw.pdf"

*Output gap and potential output charts*

In [9]:
trace1 = scatter(x=baseline_output_gap_df[:, 1], y=baseline_output_gap_df[!, Symbol(last_common_release)], line=attr(width=1.4, color=c1), mode="lines", showlegend=true, name="Baseline");
trace2 = scatter(x=restricted_output_gap_df[:, 1], y=restricted_output_gap_df[!, Symbol(last_common_release)], line=attr(width=1.4, color=c2), mode="lines", showlegend=true, name="Restricted");

layout = Layout(title="Output gap (as at $(Dates.format(last_common_release, "dd/mm/yyyy")))", titlefont_size=12,
                xaxis=attr(tickfont_size=10, showgrid=true, linecolor="black", mirror=true, nticks=10, tickangle=0, titlefont=attr(size=10)),
                yaxis=attr(zeroline=true, tickfont_size=10, showgrid=true, linecolor="black", nticks=10, mirror=true, titlefont=attr(size=10), title="Percent"));
        
fig = plot([trace1, trace2], layout);

# Size
fig.plot.layout["width"]  = 1000;
fig.plot.layout["height"] = 400;

# Margins
fig.plot.layout["margin"][:b]  = 40;
fig.plot.layout["margin"][:t]  = 40;
fig.plot.layout["margin"][:r]  = 40;
fig.plot.layout["margin"][:l]  = 40;

fig.plot.layout["legend"] = attr(y=-0.11, x=0.416, font=attr(size=10), orientation="h")

PlotlyBase.savefig(fig, "./annex/output_gap_comparison.pdf", format="pdf")

"./annex/output_gap_comparison.pdf"

In [10]:
trace1 = scatter(x=baseline_potential_yoy_df[:, 1], y=baseline_potential_yoy_df[!, Symbol(last_common_release)], line=attr(width=1.4, color=c1), mode="lines", showlegend=true, name="Baseline");
trace2 = scatter(x=restricted_potential_yoy_df[:, 1], y=restricted_potential_yoy_df[!, Symbol(last_common_release)], line=attr(width=1.4, color=c2), mode="lines", showlegend=true, name="Restricted");

layout = Layout(title="Potential output (YoY, % - as at $(Dates.format(last_common_release, "dd/mm/yyyy")))", titlefont_size=12,
                xaxis=attr(tickfont_size=10, showgrid=true, linecolor="black", mirror=true, nticks=10, tickangle=0, titlefont=attr(size=10)),
                yaxis=attr(zeroline=false, tickfont_size=10, showgrid=true, linecolor="black", nticks=10, mirror=true, titlefont=attr(size=10), title="Percent"));
        
fig = plot([trace1, trace2], layout);

# Size
fig.plot.layout["width"]  = 1000;
fig.plot.layout["height"] = 400;

# Margins
fig.plot.layout["margin"][:b]  = 40;
fig.plot.layout["margin"][:t]  = 40;
fig.plot.layout["margin"][:r]  = 40;
fig.plot.layout["margin"][:l]  = 40;

fig.plot.layout["legend"] = attr(y=-0.11, x=0.416, font=attr(size=10), orientation="h")

PlotlyBase.savefig(fig, "./annex/potential_yoy_comparison.pdf", format="pdf")

"./annex/potential_yoy_comparison.pdf"

*One-step ahead forecast charts*

In [11]:
trace1 = scatter(x=gdp_forecast_df[!, :releases], y=gdp_forecast_df[!, :baseline], line=attr(width=1.4, color=c1), mode="lines", showlegend=true, name="Baseline");
trace2 = scatter(x=gdp_forecast_df[!, :releases], y=gdp_forecast_df[!, :restricted], line=attr(width=1.4, color=c2), mode="lines", showlegend=true, name="Restricted");
trace3 = scatter(x=gdp_forecast_df[!, :releases], y=gdp_forecast_df[!, :outturn], line=attr(width=1.4, color="black", dash="dot"), mode="lines", showlegend=true, name="Outturn");
    
layout = Layout(title="Real GDP (one quarter ahead forecast)", titlefont_size=12,
                xaxis=attr(tickfont_size=10, showgrid=true, linecolor="black", mirror=true, nticks=10, tickangle=0, range=[minimum(gdp_forecast_df[!, :releases]), maximum(gdp_forecast_df[!, :releases])], titlefont=attr(size=10), title="Releases"),
                yaxis=attr(zeroline=false, tickfont_size=10, showgrid=true, linecolor="black", nticks=10, mirror=true, titlefont=attr(size=10), title="Bil. Chn. 2012\$"));
        
fig = plot([trace1, trace2, trace3], layout);

# Size
fig.plot.layout["width"]  = 1000;
fig.plot.layout["height"] = 400;

# Margins
fig.plot.layout["margin"][:b]  = 40;
fig.plot.layout["margin"][:t]  = 40;
fig.plot.layout["margin"][:r]  = 40;
fig.plot.layout["margin"][:l]  = 40;

fig.plot.layout["legend"] = attr(y=-0.22, x=0.35, font=attr(size=10), orientation="h")

PlotlyBase.savefig(fig, "./annex/gdp_forecast.pdf", format="pdf")

"./annex/gdp_forecast.pdf"

In [12]:
trace1 = scatter(x=headline_forecast_df[!, :releases], y=headline_forecast_df[!, :baseline], line=attr(width=1.4, color=c1), mode="lines", showlegend=true, name="Baseline");
trace2 = scatter(x=headline_forecast_df[!, :releases], y=headline_forecast_df[!, :restricted], line=attr(width=1.4, color=c2), mode="lines", showlegend=true, name="Restricted");
trace3 = scatter(x=headline_forecast_df[!, :releases], y=headline_forecast_df[!, :outturn], line=attr(width=1.4, color="black", dash="dot"), mode="lines", showlegend=true, name="Outturn");
    
layout = Layout(title="Headline inflation (YoY, % - one month ahead forecast)", titlefont_size=12,
                xaxis=attr(tickfont_size=10, showgrid=true, linecolor="black", mirror=true, nticks=10, tickangle=0, range=[minimum(headline_forecast_df[!, :releases]), maximum(headline_forecast_df[!, :releases])], titlefont=attr(size=10), title="Releases"),
                yaxis=attr(zeroline=false, tickfont_size=10, showgrid=true, linecolor="black", nticks=10, mirror=true, titlefont=attr(size=10), title="Percent"));
        
fig = plot([trace1, trace2, trace3], layout);

# Size
fig.plot.layout["width"]  = 1000;
fig.plot.layout["height"] = 400;

# Margins
fig.plot.layout["margin"][:b]  = 40;
fig.plot.layout["margin"][:t]  = 40;
fig.plot.layout["margin"][:r]  = 40;
fig.plot.layout["margin"][:l]  = 40;

fig.plot.layout["legend"] = attr(y=-0.22, x=0.35, font=attr(size=10), orientation="h")

PlotlyBase.savefig(fig, "./annex/headline_forecast.pdf", format="pdf")

"./annex/headline_forecast.pdf"